In [77]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [78]:
df = pd.read_csv(r'C:\Users\Матвей\Salary_Data.csv')
df.head(3)

,YearsExperience,Age,Salary
0,1.1,21.0,39343
1,1.3,21.5,46205
2,1.5,21.7,37731


In [79]:
df.shape

(30, 3)

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YearsExperience  30 non-null     float64
 1   Age              30 non-null     float64
 2   Salary           30 non-null     int64  
dtypes: float64(2), int64(1)
memory usage: 848.0 bytes


In [81]:
df['Salary'] = df['Salary'].astype(np.float32)

In [82]:
from sklearn.model_selection import train_test_split

features = ['YearsExperience', 'Age']
target = 'Salary'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            shuffle=True,
            test_size=0.3,
            random_state=42
        )

In [83]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline


class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

YearsExperience = Pipeline([
                ('ohe', OHEEncoder(key='YearsExperience'))
            ])

In [84]:
YearsExperience.fit_transform(X_train, y_train)

,YearsExperience,Age
0,1.1,21.0
4,2.2,22.2
16,5.1,26.0
5,2.9,23.0
13,4.1,24.0
11,4.0,24.0
22,7.9,31.0
1,1.3,21.5
2,1.5,21.7
25,9.0,34.0


In [85]:
from sklearn.preprocessing import StandardScaler

YearsExperience =  Pipeline([
                ('selector', NumberSelector(key='YearsExperience')),
                ('standard', StandardScaler())
            ])

In [86]:
from sklearn.preprocessing import StandardScaler

Age =  Pipeline([
                ('selector', NumberSelector(key='Age')),
                ('standard', StandardScaler())
            ])

In [87]:
feats = FeatureUnion([('Age', Age)])
feature_processing = Pipeline([('feats', feats)])

In [88]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Age')),
                                                                 ('standard',
                                                                  StandardScaler())]))])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [89]:
y_pred = pipeline.predict_proba(X_test)
y_pred

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.01      , 0.03      , 0.07      , 0.62      ,
        0.27      ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.18483333, 0.        , 0.13516667, 0.        ,
        0.68      , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.18016667,
        0.        , 0.14983333, 0.63      , 0.04      , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.005     , 0.        , 0.005    

In [100]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(max_depth=6, 
                               max_features=0.5, 
                               n_estimators=100,
                               random_state=42)

In [101]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from  sklearn.metrics import r2_score
kf = KFold(n_splits=5, random_state=42, shuffle=True)

cv_score = cross_val_score(forest, X, y, scoring='r2', cv=kf)
print('R2: {:.5f}'.format(cv_score.mean()))

R2: 0.92555


In [102]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)

In [103]:
import urllib.request
import json      

def get_prediction(YearsExperience, Age):
    body = {'YearsExperience': YearsExperience, 
                            'Age': Age} 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [106]:
get_prediction('Stylect is a dynamic startup that h', 'offers', 'free ')

URLError: <urlopen error [WinError 10049] Требуемый адрес для своего контекста неверен>